**1. Phân tích và tiền xử lý dữ liệu**

In [ ]:
from zipfile import ZipFile
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 as cv

In [ ]:
#Lấy tên của các folder làm labels
facial_age_path = "../input/facial-age/face_age"
facial_age_folders = os.listdir(facial_age_path)

In [ ]:
#Xóa tên folder bị thừa
facial_age_folders.remove('face_age')
print(facial_age_folders)

In [ ]:
#Lấy ảnh trong mỗi folder rồi lưu vào trong dictionary với labels là keys và số lượng ảnh của mỗi label là values
facial_age_images = {}

#Đối với từng tên folder 
for age in facial_age_folders:
    #Truy cập tới folder 
    temp_path = os.path.join(facial_age_path, age)
    #Lấy số lượng ảnh trong folder
    n_images = len(os.listdir(temp_path))
    facial_age_images[int(age)] = n_images

In [ ]:
#Kiểm tra lại số lượng ảnh (số lượng ảnh của facial-age là 9778)
sum(facial_age_images.values())

In [ ]:
#Lấy tên của các folder làm labels
utkface_path = '../input/utkface-new/UTKFace'
utkface_image_names = os.listdir(utkface_path)

In [ ]:
#Kiểm tra lại số lượng ảnh 
len(utkface_image_names)

In [ ]:
#Do chỉ sử dụng thông tin AGE của dataset nên ta sẽ tách tên của mỗi ảnh ra và chỉ giữ lại thôn tin AGE của mỗi ảnh
utkface_age_labels = np.array([])

for image in utkface_image_names:
    image_labels = image.split('_')
    age = image_labels[0]
    utkface_age_labels = np.append(utkface_age_labels, age) 

In [ ]:
#Nhóm lại các ảnh theo từng độ tuổi sau đó lấy ảnh trong mỗi folder rồi lưu vào trong dictionary với labels là keys và số lượng ảnh của mỗi label là values

utkface_images = {}
#Nhóm lại theo từng label tuổi
utkface_ages_counts = pd.Series(utkface_age_labels).value_counts()
for age, counts in utkface_ages_counts.items():
    utkface_images[int(age)] = counts

**2. Phân lớp dữ liệu**

In [ ]:
#Kiểm tra lại số lượng ảnh (số lượng ảnh của utkface là 23708)
sum(utkface_images.values())

In [ ]:
#Kiểm tra số lượng tuổi khác nhau trong bộ facial-age
facial_age_ages = list(facial_age_images.keys())
len(facial_age_ages)

In [ ]:
#Kiểm tra số lượng tuổi khác nhau trong bộ utkface
utkface_ages = list(utkface_images.keys())
len(utkface_ages)

In [ ]:
#Kết hợp hai dataset lại thành một và chuyển từ list về set chỉ giữ các giá trị tuổi unique
facial_age_ages.extend(utkface_ages)
unique_ages = set(facial_age_ages)

In [ ]:
#Tạo ra dictionary mới được kết hợp từ 2 dictionary ở trên
combined_images = {}

for age in unique_ages:
    fc_image = 0
    utk_image = 0

    #Sử dụng vòng lặp try-except để tránh KeyError trong trường hợp giá trị tuổi không xuất hiện trong dictionary
    try:
        fc_image = facial_age_images[age]
    except:
        pass
    
    try:
        utk_image = utkface_images[age]
    except:
        pass
  
    combined_images[age] = fc_image + utk_image

In [ ]:
#Kiểm tra lại số lượng ảnh (số lượng ảnh của combined_images là 33486)
sum(combined_images.values())

In [ ]:
#Tạo ra một dataframe để chứa số lượng của từng tuổi
images_df = pd.DataFrame(combined_images.values(), index=combined_images.keys(), columns=['combined_images'])
images_df['facial_age_images'] = pd.Series(facial_age_images)
images_df['utkface_images'] = pd.Series(utkface_images)

images_df

In [ ]:
#Sửa lại các giá trị bị null thành dạng số 0 và đổi các giá trị về dạng integer
images_df.fillna(0, inplace=True)
images_df = images_df.astype(int)

images_df

In [ ]:
#Xuất ra file csv để train model với trường hợp label là tất cả các độ tuổi
images_df.to_csv("images_summary.csv", index=True, index_label='age')

In [ ]:
#Đoạn cell để show số lượng ảnh của từng độ tuổi trong Combined datasets
plt.figure(figsize=(30, 10))

ax = sns.barplot(x=images_df.index, y=images_df['combined_images'], color='royalblue')

ax.tick_params(axis='both', labelsize=12)
ax.tick_params(axis='x', labelrotation=45)

plt.xlabel("Tuổi", fontsize=16)
plt.ylabel("Số lượng ảnh", fontsize=16)

plt.title("Barplot biểu diễn số lượng ảnh của từng độ tuổi trong Combined datasets", fontsize=18)

Từ biểu đồ trên ta thấy rằng có sự chênh lệch lớn về số lượng ảnh giữa các độ tuổi cộng thêm việc có quá nhiều độ tuổi được gán nhãn sẽ dẫn đến việc làm giảm hiệu quả dự đoán tuổi của model. Nhóm đã thực nghiệm và thấy rằng việc này sẽ dẫn đến hậu quả model bị overfitting. Kết quả sẽ được show trong slides

Vậy nên nhóm thấy rằng phân chia độ tuổi lại thành các nhóm tuổi là việc làm cần thiết

In [ ]:
#Tạo hàm để chia dữ liệu tuổi trong dataset thành n khoảng(lớp)
#Input: Một cột trong dataframe độ tuổi và số lượng lớp muốn chia
#Output: Dataset đã được chia thành n lớp, đi kèm là thông tin về khoảng tuổi, số lượng ảnh và độ cân bằng trong từng khoảng
def split_classes(ser, n_classes): 
    n_images = int(sum(ser) / n_classes)
    classes_df = pd.DataFrame(columns=['Age-ranges (classes)', 'No. of images', 'Class balance (%)'])
    age_index = 0

    for i in range(n_classes):
        if age_index<=103:
            age_start = ser.index[age_index]
            age_current = ser.index[age_index]
        else:
            break

        class_images = 0
        
        while class_images < n_images:
            class_images += ser[age_current]
            age_index += 1
            if age_index<=103:
                age_current = ser.index[age_index]
            else:
                break

        if age_index<=104:
            age_end = ser.index[age_index-1]
        else:
            break
        
        classes_df.loc[i, 'Age-ranges (classes)'] = str(age_start)+" - "+str(age_end)
        classes_df.loc[i, 'No. of images'] = class_images
        classes_df.loc[i, 'Class balance (%)'] = round((class_images / sum(ser)) * 100, 2)

    return classes_df

In [ ]:
#Chia bộ combined_images ra thành 7 lớp
combined_classes = split_classes(images_df['combined_images'], 7)
combined_classes

In [ ]:
#Lưu lại thành file csv
combined_classes.to_csv("combined_faces_classes_summary.csv", index=True, index_label='Class label')

In [ ]:
#Tạo folder tên combined_faces
os.mkdir("combined_faces")

progress_counter = 0
age_file_counter = [1] * 117

print("Merging ảnh từ bộ facial-age sang combined_faces.\n")
for age in facial_age_folders:
    age_path = os.path.join(facial_age_path, age)

    img_files = os.listdir(age_path)

    for img in img_files:

        img_src = os.path.join(age_path, img)
        
        #Chuyển ảnh về dạng JPG cho giống bộ utkface
        new_filename = str(int(age)) + "_" + str(age_file_counter[int(age)]) + ".jpg"
        age_file_counter[int(age)] += 1

        img_dest = os.path.join("/kaggle/working/combined_faces", new_filename)

        png_image = cv.imread(img_src)
        cv.imwrite(img_dest, png_image, [int(cv.IMWRITE_JPEG_QUALITY), 100])

        progress_counter += 1

        if progress_counter % 1000 == 0:
            print(f"Images copied to combined_faces folder: {progress_counter} of 33486")

print("\nMerging ảnh từ bộ UTKFace sang combined_faces.\n")

for img in utkface_image_names:

    file_type = img.split(".")[-1]
    age = img.split("_")[0]

    img_src = os.path.join(utkface_path, img)

    new_filename = age + "_" + str(age_file_counter[int(age)]) + "." + file_type
    age_file_counter[int(age)] += 1

    img_dest = os.path.join("/kaggle/working/combined_faces", new_filename)

    shutil.copy(img_src, img_dest);
    progress_counter += 1

    if progress_counter % 1000 == 0:
        print(f"Images copied to combined_faces folder: {progress_counter} of 33486")
        
print("\nHoàn thành")

**3. Phân chia dataset thành bộ Train và bộ Test**

In [ ]:
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.feature import canny

import time
from datetime import datetime

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
#Truy cập tới folder ảnh
combined_faces_path = "/kaggle/working/combined_faces"
combined_faces_image_names = os.listdir(combined_faces_path)

In [ ]:
len(combined_faces_image_names)

In [ ]:
#dataframe 8 label khoảng tuổi
combined_classes = pd.read_csv("/kaggle/working/combined_faces_classes_summary.csv")

#dataframe 116 label tất cả độ tuổi
#combined_classes = pd.read_csv("/kaggle/working/images_summary.csv")

combined_classes

In [ ]:
#Tạo hàm trả về các lớp label dựa trên các khoảng tuổi đã chia
def class_labels(age):
    if 1 <= age <= 6:
        return 0
    elif 7 <= age <= 22:
        return 1
    elif 23 <= age <= 26:
        return 2
    elif 27 <= age <= 33:
        return 3
    elif 34 <= age <= 45:
        return 4
    elif 46 <= age <= 61:
        return 5
    else:
        return 6
    
    #Dùng cho trường hợp 116 label
    #for i in range(117):
    #    if(age == i):
    #        return i-1

In [ ]:
#Tạo dataframe chứa tất cả các tên ảnh đi kèm là thông tin tuổi là lớp label
master_df = pd.DataFrame()
master_df['filename'] = combined_faces_image_names
master_df['age'] = master_df['filename'].map(lambda img_name : np.uint8(img_name.split("_")[0]))
master_df['target'] = master_df['age'].map(class_labels)

master_df.head()

In [ ]:
#Xáo trộn vị trí trong dataframe
master_df = shuffle(master_df, random_state=42).reset_index(drop=True)
master_df.head()

In [ ]:
#Khởi tạo X và y để chuẩn bị chia dataset
X = master_df[['filename', 'age']]
y = master_df['target']

In [ ]:
#Chia dataset thành 2 bộ Train(80%) và Test(20%) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
#Copy để tạo 1 dataframe của các đường dẫn file tới tất cả các ảnh và nhãn target tương ứng
temp_X_train = X_train.copy()
temp_X_train['target'] = y_train

temp_X_test = X_test.copy()
temp_X_test['target'] = y_test

In [ ]:
#Tạo hàm thêm đường dẫn tới file tương ứng mỗi ảnh 
combined_faces_path = "/kaggle/working/combined_faces"

def append_path_to_filename(filename):
    return os.path.join(combined_faces_path, filename)

In [ ]:
temp_X_train['filename'] = temp_X_train['filename'].map(append_path_to_filename)
temp_X_test['filename'] = temp_X_test['filename'].map(append_path_to_filename)

In [ ]:
temp_X_train.to_csv("images_filenames_labels_train.csv", index=False)
temp_X_test.to_csv("images_filenames_labels_test.csv", index=False)

**4. Trích xuất đặc trưng sử dụng Canny**

In [ ]:
# Hàm để chia hình ảnh size 200x200px thành các sections size 10x10px và tính giá trị mean và stdev cho từng section
# INPUT: Một ảnh size 200x200px
# OUTPUT: Mảng đặc trưng gồm giá trị mean và stdev của 400 sections

def features_grid(img):
    features = np.array([], dtype='uint8')
    section = 1
    
    for y in range(0, img.shape[0], 10):
    #for y in range(0, img.shape[0], 20):
        for x in range(0, img.shape[1], 10):
        #for x in range(0, img.shape[1], 20):

            # Cắt ảnh thành 1 section
            #size 10x10
            section_img = img[y:y+10, x:x+10]
            
            #size 20x20
            #section_img = img[y:y+20, x:x+20]
            
            # Tính giá trị mean và std cho section
            section_mean = np.mean(section_img)
            section_std = np.std(section_img)
            
            # Thêm 2 giá trị vừa tính và mảng đặc trưng
            features = np.append(features, [section_mean, section_std])
    
    # Trả về mảng đặc trưng
    return features

In [ ]:
# Hàm để đọc tất cả ảnh trong dataset và trích xuất đặc trưng cạnh Canny với 2 giá trị mean và stdev trên từng section mỗi ảnh
def extract_canny_edges(filename_series):

    # Tạo một mảng để chứa 400 giá trị mean, 400 giá trị stdev của đặc trưng cạnh Canny và 1 giá trị tuổi
    all_imgs = np.zeros((1, 801), dtype='uint8')
    
    # Tạo một mảng để chứa 100 giá trị mean, 100 giá trị stdev của đặc trưng cạnh Canny và 1 giá trị tuổi
    #all_imgs = np.zeros((1, 201), dtype='uint8')

    progress_counter = 0

    for img_name in filename_series:
        img_path = os.path.join(combined_faces_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        img = canny(img, sigma=0.9)
        
        img_features = features_grid(img)

        # Thêm giá trị tuổi thật (từ tên ảnh) vào mảng giá trị 
        age = np.uint8(img_name.split("_")[0])
        img_features = np.append(img_features, age)

        img_features = img_features.reshape(1, img_features.shape[0])

        # Thêm đặc trưng ảnh hiện tại vào mảng đặc trưng trưng toàn dataset
        all_imgs = np.append(all_imgs, img_features, axis=0)

        progress_counter += 1
        if progress_counter % 1000 == 0:
            print(f"Images processed for features extraction: {progress_counter} of {len(filename_series)}")

    # Xóa cột giá trị 0 đầu tiên khi tạo mảng đặc trưng
    all_imgs = all_imgs[1:]

    return all_imgs

In [ ]:
train_imgs = extract_canny_edges(X_train['filename'])

In [ ]:
with open("/kaggle/working/canny_features_age_train.npy", "wb") as f:
    np.save(f, train_imgs, allow_pickle=True)

In [ ]:
test_imgs = extract_canny_edges(X_test['filename'])

In [ ]:
with open("/kaggle/working/canny_features_age_test.npy", "wb") as f:
    np.save(f, test_imgs, allow_pickle=True)

In [ ]:
# Tạo một danh sách tên cột cho các mảng đặc trưng tạo ở trên
# Các tên cột dựa trên tên các giá trị mean và stdev trên từng section ảnh
# Cột cuổi chứa các nhãn target class

feature_names = []
section = 1
    
for y in range(0, 200, 10):
    for x in range(0, 200, 10):
        feature_names.append(f"sec{section}_mean")
        feature_names.append(f"sec{section}_std")
        section += 1

feature_names.append('age')

In [ ]:
pd.Series(feature_names).to_csv("/kaggle/working/canny_features_names.csv", index=False, header=['canny_edge_features'])

**5. Huấn luyện mô hình**

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

import itertools

import pickle

In [ ]:
combined_classes = pd.read_csv("/kaggle/working/combined_faces_classes_summary.csv")
#combined_classes = pd.read_csv("/kaggle/working/images_summary.csv")
combined_classes

In [ ]:
feature_names = pd.read_csv("/kaggle/working/canny_features_names.csv")
feature_names

In [ ]:
train = np.load("/kaggle/working/canny_features_age_train.npy")
test = np.load("/kaggle/working/canny_features_age_test.npy")

In [ ]:
train_df = pd.DataFrame(train, columns=feature_names["canny_edge_features"])
test_df = pd.DataFrame(test, columns=feature_names["canny_edge_features"])

In [ ]:
train_df['age'] = train_df['age'].astype(np.uint8)
test_df['age'] = test_df['age'].astype(np.uint8)

In [ ]:
train_df['target'] = train_df['age'].map(class_labels)
test_df['target'] = test_df['age'].map(class_labels)

In [ ]:
X_train = train_df.drop(columns=['age', 'target'])
y_train = train_df['target']

X_test = test_df.drop(columns=['age', 'target'])
y_test = test_df['target']

In [ ]:
y_train.value_counts()

In [ ]:
svc = SVC(
          kernel='rbf',
         )

In [ ]:
svc.fit(X_train, y_train)

In [ ]:
svc_train_acc = svc.score(X_train, y_train)

In [ ]:
svc_test_acc = svc.score(X_test, y_test)

In [ ]:
print("SVC summary of accuracy scores:")
print(f"Training accuracy = {round(svc_train_acc, 3)}")
print(f"Testing accuracy = {round(svc_test_acc, 3)}")

In [ ]:
svc_pred = svc.predict(X_test)

In [ ]:
conf_mat_svc = confusion_matrix(y_test, svc_pred)
conf_mat_svc

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', export_as='confusion_matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    # print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=16)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True labels', fontsize=14)
    plt.xlabel('Predicted labels', fontsize=14)



In [ ]:
cm_plot_labels = combined_classes['Age-ranges (classes)']
plt.figure(figsize=(16,8))
plot_confusion_matrix(conf_mat_svc, cm_plot_labels, normalize=True,
                      title="Confusion Matrix based on predictions from\nSVC model using Canny Edge features",
                      export_as="svc_canny_conf_mat_norm"
                     )

plt.show()